In [ ]:
import torch

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdriv')

Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


In [ ]:
#nsmc 다운로드
!git clone https://github.com/e9t/nsmc.git  #영화 댓글 데이터 

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [ ]:
import pandas as pd

train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)
## 리뷰 문장 추출
#sentences = train['document']
#라벨 추출
#labels = train['label'].values
train.drop(['id'],inplace=True,axis=1)
train.rename(columns={'document':'review','sentiment':'label'},inplace=True)

(150000, 3)
(50000, 3)


In [ ]:
train_set=train[:1000]
valid_set=train[1000:1500]
test_set=train[1500:2000]

In [ ]:
kb_albert_model_path='/content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/kb-albert-char-base-v2'

In [ ]:
'''필사한 건데 안 돌아ㅠㅠ

from transformers import AutoTokenizer

class TheDataset(torch.utils.data.Dataset):

  def __init__(self,reviews,sentiments,tokenizer):
    self.reviews=reviews
    self.sentiments=sentiments
    self.tokenizer=tokenizer
    self.max_len=tokenizer.model_max_length

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self,index):
    review=str(self.reviews[index])
    sentiments=self.sentiments[index]

    encoded_review=self.tokenzier.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=True,
        return_tensors='pt',
        padding='max_length',
        truncation=True
    )



    return{
        'input_ids':encoded_review['input_ids'][0],
        'attention_mask':encoded_review['attention_mask'][0],
        'labels':torch.tensor(sentiments,dtype=torch.long)
    }

tokenizer=AutoTokenizer.from_pretrained(kb_albert_model_path)

train_set_dataset=TheDataset(
    reviews=train_set.review.tolist(),
    #sentiments=train_set.vader_result.tolist(),  #vader 라는 비지도학습기반 라벨 부여
    sentiments=train_set.label.tolist(),
    tokenizer=tokenizer,
)

valid_set_dataset=TheDataset(
    reviews=valid_set.review.tolist(),
    #sentiments=valid_set.vader_result.tolist()
    sentiments=valid_set.label.tolist(),
    tokenizer=tokenizer,
)

train_set_dataloader=torch.utils.data.DataLoader(
    train_set_dataset,
    batch_size=16,
    num_workers=2   #권장되는 코어수
)

valid_set_dataloader=torch.utils.data.DataLoader(
    valid_set_dataset,
    batch_size=16,
    num_workers=2
)

train_data=next(iter(train_set_dataloader))
valid_data=next(iter(valid_set_dataloader)) #iter를 하나씩 next하면서 집어넣음

print(train_data['input_ids'].size(),valid_data['input_ids'].size())

'''

"필사한 건데 안 돌아ㅠㅠ\n\nfrom transformers import AutoTokenizer\n\nclass TheDataset(torch.utils.data.Dataset):\n\n  def __init__(self,reviews,sentiments,tokenizer):\n    self.reviews=reviews\n    self.sentiments=sentiments\n    self.tokenizer=tokenizer\n    self.max_len=tokenizer.model_max_length\n\n  def __len__(self):\n    return len(self.reviews)\n\n  def __getitem__(self,index):\n    review=str(self.reviews[index])\n    sentiments=self.sentiments[index]\n\n    encoded_review=self.tokenzier.encode_plus(\n        review,\n        add_special_tokens=True,\n        max_length=self.max_len,\n        return_token_type_ids=True,\n        return_tensors='pt',\n        padding='max_length',\n        truncation=True\n    )\n\n\n\n    return{\n        'input_ids':encoded_review['input_ids'][0],\n        'attention_mask':encoded_review['attention_mask'][0],\n        'labels':torch.tensor(sentiments,dtype=torch.long)\n    }\n\ntokenizer=AutoTokenizer.from_pretrained(kb_albert_model_path)\n\ntrain_set_

In [ ]:
#ex1에 훈련모델 만들 때 사용한 코드
#input값인 train_set_dataset을 cuda에 올리지 않았음. 근데 왜 속도가 빨라졌지? 데이터로더에서 pin_memory=True 가 디폴트 값이어서 그런가?


from transformers import AutoTokenizer


# Create The Dataset Class.
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = self.max_len,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = "pt",
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Load the tokenizer for the BERT model.
tokenizer=AutoTokenizer.from_pretrained(kb_albert_model_path)

# Create Dataset objects for train/validation sets.
train_set_dataset = TheDataset(
    reviews    = train_set.review.tolist(),
    sentiments = train_set.label.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    reviews    = valid_set.review.tolist(),
    sentiments = valid_set.label.tolist(),
    tokenizer  = tokenizer,
)
'''
# Create DataLoader for train/validation sets.
train_set_dataloader = torch.utils.data.DataLoader(
    train_set_dataset,
    batch_size  = 16,
    num_workers = 2,
)

valid_set_dataloader = torch.utils.data.DataLoader(
    valid_set_dataset,
    batch_size  = 16,
    num_workers = 2,  #2 권장
)




# Get one batch as example.
train_data = next(iter(train_set_dataloader))
valid_data = next(iter(valid_set_dataloader))

train_data['attention_mask']=train_data['attention_mask'].to(device)
train_data['input_ids']=train_data['input_ids'].to(device)
train_data['labels']=train_data['labels'].to(device)
valid_data['attention_mask']=valid_data['attention_mask'].to(device)
valid_data['input_ids']=valid_data['input_ids'].to(device)
valid_data['labels']=valid_data['labels'].to(device)

# Print the output sizes.
print( train_data["input_ids"].size(), valid_data["input_ids"].size() )
'''

'\n# Create DataLoader for train/validation sets.\ntrain_set_dataloader = torch.utils.data.DataLoader(\n    train_set_dataset,\n    batch_size  = 16,\n    num_workers = 2,\n)\n\nvalid_set_dataloader = torch.utils.data.DataLoader(\n    valid_set_dataset,\n    batch_size  = 16,\n    num_workers = 2,  #2 권장\n)\n\n\n\n\n# Get one batch as example.\ntrain_data = next(iter(train_set_dataloader))\nvalid_data = next(iter(valid_set_dataloader))\n\ntrain_data[\'attention_mask\']=train_data[\'attention_mask\'].to(device)\ntrain_data[\'input_ids\']=train_data[\'input_ids\'].to(device)\ntrain_data[\'labels\']=train_data[\'labels\'].to(device)\nvalid_data[\'attention_mask\']=valid_data[\'attention_mask\'].to(device)\nvalid_data[\'input_ids\']=valid_data[\'input_ids\'].to(device)\nvalid_data[\'labels\']=valid_data[\'labels\'].to(device)\n\n# Print the output sizes.\nprint( train_data["input_ids"].size(), valid_data["input_ids"].size() )\n'

In [ ]:
from transformers import BertForSequenceClassification
model=BertForSequenceClassification.from_pretrained(kb_albert_model_path)

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/kb-albert-char-base-v2 were not used when initializing BertForSequenceClassification: ['sop_classifier.classifier.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'predictions.decoder.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'albert.embeddings.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'sop_classifier.classifier.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.qu

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
device

'cuda'

파인튜닝 위해서 모델 구조비교

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
for name, param in model.bert.named_parameters():
    print('이름은',name)
    print('파라미터는',param)
    print(param.requires_grad)

이름은 embeddings.word_embeddings.weight
파라미터는 Parameter containing:
tensor([[-0.0419,  0.0022, -0.0314,  ..., -0.0146, -0.0358, -0.0929],
        [-0.0434, -0.0115, -0.0065,  ..., -0.0273, -0.0310, -0.1084],
        [-0.0622, -0.0114, -0.0073,  ..., -0.0247, -0.0334, -0.0915],
        ...,
        [ 0.0239, -0.0083, -0.0061,  ...,  0.0176,  0.0214, -0.0763],
        [-0.0468, -0.0018,  0.0020,  ..., -0.0173, -0.0243, -0.1124],
        [-0.0185, -0.0369, -0.0623,  ..., -0.0054, -0.0412, -0.0847]],
       requires_grad=True)
True
이름은 embeddings.position_embeddings.weight
파라미터는 Parameter containing:
tensor([[ 0.0041, -0.0139, -0.0263,  ..., -0.0042, -0.0212,  0.0148],
        [ 0.0097,  0.0067, -0.0287,  ...,  0.0003,  0.0051,  0.0049],
        [ 0.0005, -0.0129, -0.0019,  ..., -0.0006,  0.0247,  0.0181],
        ...,
        [ 0.0047,  0.0026,  0.0021,  ...,  0.0092, -0.0113,  0.0185],
        [-0.0161, -0.0068,  0.0181,  ...,  0.0040, -0.0194,  0.0058],
        [ 0.0218,  0.0243,  0.0312,

In [ ]:
for name, param in model.bert.named_parameters():
    param.requires_grad = False

In [ ]:
for name, param in model.bert.named_parameters():
    print('이름은',name)
    print('파라미터는',param)
    print(param.requires_grad)

이름은 embeddings.word_embeddings.weight
파라미터는 Parameter containing:
tensor([[-0.0419,  0.0022, -0.0314,  ..., -0.0146, -0.0358, -0.0929],
        [-0.0434, -0.0115, -0.0065,  ..., -0.0273, -0.0310, -0.1084],
        [-0.0622, -0.0114, -0.0073,  ..., -0.0247, -0.0334, -0.0915],
        ...,
        [ 0.0239, -0.0083, -0.0061,  ...,  0.0176,  0.0214, -0.0763],
        [-0.0468, -0.0018,  0.0020,  ..., -0.0173, -0.0243, -0.1124],
        [-0.0185, -0.0369, -0.0623,  ..., -0.0054, -0.0412, -0.0847]],
       requires_grad=True)
True
이름은 embeddings.position_embeddings.weight
파라미터는 Parameter containing:
tensor([[ 0.0041, -0.0139, -0.0263,  ..., -0.0042, -0.0212,  0.0148],
        [ 0.0097,  0.0067, -0.0287,  ...,  0.0003,  0.0051,  0.0049],
        [ 0.0005, -0.0129, -0.0019,  ..., -0.0006,  0.0247,  0.0181],
        ...,
        [ 0.0047,  0.0026,  0.0021,  ...,  0.0092, -0.0113,  0.0185],
        [-0.0161, -0.0068,  0.0181,  ...,  0.0040, -0.0194,  0.0058],
        [ 0.0218,  0.0243,  0.0312,

In [ ]:
for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.23" not in name :
        param.requires_grad = False

In [ ]:
for name, param in model.bert.named_parameters():
    print('=====================')
    print('이름은',name)
    print('파라미터는',param)
    print(param.requires_grad)

이름은 embeddings.word_embeddings.weight
파라미터는 Parameter containing:
tensor([[-0.0419,  0.0022, -0.0314,  ..., -0.0146, -0.0358, -0.0929],
        [-0.0434, -0.0115, -0.0065,  ..., -0.0273, -0.0310, -0.1084],
        [-0.0622, -0.0114, -0.0073,  ..., -0.0247, -0.0334, -0.0915],
        ...,
        [ 0.0239, -0.0083, -0.0061,  ...,  0.0176,  0.0214, -0.0763],
        [-0.0468, -0.0018,  0.0020,  ..., -0.0173, -0.0243, -0.1124],
        [-0.0185, -0.0369, -0.0623,  ..., -0.0054, -0.0412, -0.0847]])
False
이름은 embeddings.position_embeddings.weight
파라미터는 Parameter containing:
tensor([[ 0.0041, -0.0139, -0.0263,  ..., -0.0042, -0.0212,  0.0148],
        [ 0.0097,  0.0067, -0.0287,  ...,  0.0003,  0.0051,  0.0049],
        [ 0.0005, -0.0129, -0.0019,  ..., -0.0006,  0.0247,  0.0181],
        ...,
        [ 0.0047,  0.0026,  0.0021,  ...,  0.0092, -0.0113,  0.0185],
        [-0.0161, -0.0068,  0.0181,  ...,  0.0040, -0.0194,  0.0058],
        [ 0.0218,  0.0243,  0.0312,  ...,  0.0091,  0.0013,  

In [ ]:
for n,p in model.bert.named_parameters():
  print(n)
  #print("layer.23" not in name)


embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

예제는, 여기서 풀러층과 마지막 레이어를 고정시켰음

In [ ]:
model=BertForSequenceClassification.from_pretrained(kb_albert_model_path)

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/kb-albert-char-base-v2 were not used when initializing BertForSequenceClassification: ['albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'albert.embeddings.position_ids', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'albert.pooler.bias', 'albert.embeddings.position_embeddings.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'albert.encoder.embedding_hidden_mapping_in.weight', 'predictions.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn_output.bias', 'predictions.LayerNorm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'albert.pooler.weight', 'albert.encoder.albert_layer_groups.0.albert_layer

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(9607, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
for n,p in model.bert.named_parameters():
  print(n)
  #print("layer.23" not in name)


embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [ ]:
for name, param in model.bert.named_parameters():
    print('=====================')
    print('이름은',name)
    print('파라미터는',param)
    print(param.requires_grad)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 

In [ ]:
#여기 미세조정 파트!! 중요함!! 정확도를 결정해줌

'''
 각 레이어 내에서 파라미터가 있고, 레이어마다 .param 함수를 호출해줌으로써 이를 얻을 수 있다. 
 모든 파라미터는 requires_grad라는 특정성을 가진다. 이 특성의 기본값은 True인데, 역전파를 하겠다는 말이다. 
 그래서 한 레이어를 얼리려면 requires_grad를 False로 설정할 필요가 있다.
'''

#풀러층과 마지막 레이어 제외 고정
for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.11" not in name :
        param.requires_grad = False

In [ ]:
# 풀러층빼고 고정
for name, param in model.bert.named_parameters():
    if not name.startswith('pooler'):
        param.requires_grad = False


In [ ]:
for name, param in model.bert.named_parameters():
    print('=====================')
    print('이름은',name)
    print('파라미터는',param)
    print(param.requires_grad)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 

In [ ]:
model=model.to(device) #모델 cuda에 올리기

In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
from transformers import TrainingArguments,Trainer

def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  precision,recall,f1,_=precision_recall_fscore_support(labels,preds,average='binary')
  acc=accuracy_score(labels,preds)
  return {
      'accuracy':acc,
      'f1':f1,
      'precision':precision,
      'recall':recall
  }

training_args=TrainingArguments(
    output_dir='/content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2',
    num_train_epochs=200,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.00005,
    save_strategy='epoch',
    evaluation_strategy='steps'
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set_dataset,
    eval_dataset=valid_set_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 200
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1600


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.672300,0.664151,0.574000,0.241993,0.755556,0.144068
1000,0.587400,0.622432,0.650000,0.611973,0.641860,0.584746
1500,0.539700,0.621607,0.656000,0.622807,0.645455,0.601695


Saving model checkpoint to /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-8
Configuration saved in /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-8/config.json
Model weights saved in /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-8/pytorch_model.bin
Saving model checkpoint to /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-16
Configuration saved in /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-16/config.json
Model weights saved in /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-16/pytorch_model.bin
Saving model checkpoint to /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-24
Configuration saved in /content/gdriv/My D

TrainOutput(global_step=1600, training_loss=0.5953432965278626, metrics={'train_runtime': 7023.6916, 'train_samples_per_second': 28.475, 'train_steps_per_second': 0.228, 'total_flos': 5.2622211072e+16, 'train_loss': 0.5953432965278626, 'epoch': 200.0})

#체크포인트 로드 및 예측

In [ ]:
model=BertForSequenceClassification.from_pretrained('/content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-1600')

test_set_dataset=TheDataset(
    reviews=test_set.review.tolist(),
    sentiments=test_set.label.tolist(),
    tokenizer=tokenizer,
)

training_args=TrainingArguments(
    output_dir='/content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2',
    do_predict=True
)

trainer=Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
)

trainer.predict(test_set_dataset)



loading configuration file /content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/nsmc_outputs_ex2/checkpoint-1600/config.json
Model config BertConfig {
  "_name_or_path": "/content/gdriv/My Drive/Colab Notebooks/KB-albert/kb-albert-char-base-v2/kb-albert-char-base-v2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout": null,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",


PredictionOutput(predictions=array([[-3.95171642e-01,  4.59008425e-01],
       [ 2.18864486e-01, -2.44162723e-01],
       [-4.44268942e-01,  4.94385958e-01],
       [-1.33445120e+00,  1.48470736e+00],
       [-1.33104965e-01,  1.66702136e-01],
       [-7.22259805e-02,  1.04851775e-01],
       [-8.58000517e-01,  9.71905529e-01],
       [ 5.10776639e-01, -4.23061937e-01],
       [ 1.07608186e-02, -4.71698912e-03],
       [-1.57481641e-01,  2.83212811e-01],
       [ 1.57623872e-01, -1.05107993e-01],
       [ 1.81896761e-01, -5.65374047e-02],
       [-1.92913234e-01,  4.23656762e-01],
       [-3.67724933e-02,  9.49883014e-02],
       [-8.18463191e-02,  2.12867498e-01],
       [ 3.03513914e-01, -2.28120327e-01],
       [-3.13436682e-03,  9.04938206e-02],
       [ 5.08770823e-01, -4.37584817e-01],
       [-3.36285055e-01,  3.13033313e-01],
       [-2.24346817e-02,  1.33023962e-01],
       [-5.06151736e-01,  4.92213815e-01],
       [-1.38960019e-01,  1.66828498e-01],
       [-3.09725434e-01, 